In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# pip install -U scikit-learn

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [5]:
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4391885 sha256=256f174b75506dc9a9e71b65dac789c524315167e3d6a8f5137f1b3cab8b9083
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [6]:
df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)
df.sample(5)

<ipython-input-6-7fde084fdf7a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)


,text,class,char_length,token_length
45425,pm bored want talk pm favourite song atm low,0,54,11
115292,realized insane parent voice sex appeal alarm ...,0,106,14
68962,need escape narcissistic abusive mother moment...,1,1111,153
177678,hello fellow kid need input youtube tip fokker...,0,790,85
90809,week wont return college anymore live got seda...,1,68,11


In [7]:
model = fasttext.load_model('drive/MyDrive/TextAnalytics/cc.en.300.bin')
def get_text_vector(sentence): return model.get_sentence_vector(sentence)

In [8]:
df['text_vectors'] = df['text'].apply(get_text_vector)
df.sample(5)

,text,class,char_length,token_length,text_vectors
17301,imagine scenario string room drinking tea havi...,0,86,12,"[-0.0036319632, 0.022356328, -0.0035427809, 0...."
44897,racist white people euuuuggggghhhaaa,0,36,4,"[0.015894776, -0.040982123, 0.0007186569, 0.10..."
200289,sub dead like responds post anymore come boy l...,0,125,19,"[0.033448454, 0.021455158, 0.012036234, 0.0863..."
16621,weave fucked wall street let fuck nestle nestl...,0,402,57,"[0.0014603926, -0.012301582, 0.0222896, 0.0811..."
55511,fuck gained weight kinda skinny year started g...,0,90,13,"[-0.0015458678, -0.0053919214, 0.024639234, 0...."


In [9]:
X, y = df['text_vectors'], df['class']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [11]:
clf=XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=500, objective='binary:logistic', booster='gbtree')

In [12]:
### pending parameter tuning here...

In [13]:
#Creating the model on Training Data
XGP_model = clf.fit(list(X_train),y_train)

In [14]:
y_pred = XGP_model.predict(list(X_test))

#Confusion Matrix
clf_tn, clf_fp, clf_fn, clf_tp = confusion_matrix(y_test, y_pred).ravel()
#measurements
clf_accuracy = (clf_tp+clf_tn)/(clf_tp+clf_tn+clf_fp+clf_fn)
print(" Accuracy", round(clf_accuracy,4))
clf_precision = clf_tp/(clf_tp+clf_fp)
clf_recall = clf_tp/(clf_tp+clf_fn)
clf_f1_score = (2*clf_precision)*(clf_recall)/(clf_precision+clf_recall)
print(" Precision",round(clf_precision,4),"\n","Recall",round(clf_recall,4),"\n","F1",round(clf_f1_score,4))

 Accuracy 0.9112
 Precision 0.914 
 Recall 0.9088 
 F1 0.9114
